In [1]:
using BenchmarkTools: @btime
import LinearAlgebra as la
using TensorOperations: @tensor

path = "../../src/"
include(path * "Systems/system.jl")
include(path * "Methods/HF.jl")
include(path * "Methods/CCD.jl")
include(path * "Methods/CCSD.jl")
include(path * "Methods/mixer.jl")

compute_new_vector (generic function with 1 method)

In [2]:
using PyCall

In [3]:
py"""
import numpy as np
import scipy.special as spec
"""
@pyinclude("two_body_integrals.py")
@pyinclude("transform.py")
@pyinclude("HF.py")
@pyinclude("DIIS.py")

### Setup

In [118]:
l = 3
ω = 0.25
basis = HOBasis(l, ω)

n = 2
V = ShieldedCoulomb(0.25)
grid = [x for x in range(-10, stop = 10, length = 201)]
spfs = spatial(basis, grid);

In [119]:
for i in 1:l
    if i in [2,3,6,7]
        spfs[i] = spfs[i] * -1
    end
end

In [120]:
h = onebody(basis, grid);

### Inner ints

In [121]:
my_inner = inner_ints(spfs, grid, V::Interaction);

In [122]:
spf = reduce(vcat,transpose.(spfs));
py_inner = py"_compute_inner_integral"(spf, basis.l, length(grid), Vector(grid), 1, 0.25);

In [123]:
py_inner ≈ my_inner

true

### Outer ints

In [124]:
my_u = outer_int(spfs, grid, my_inner);

In [125]:
py_u = py"_compute_orbital_integrals"(spf, basis.l, py_inner, Vector(grid));

In [126]:
py_u ≈ my_u

true

### Add spin

In [127]:
py"
def antisym(u):
    #return (u - u.transpose(0, 1, 3, 2)).real
    return u.real
"

In [128]:
my_us = add_spin_u(my_u);

In [129]:
py_us = py"add_spin_two_body"(py_u);

In [130]:
my_us ≈ py_us

true

### Transform

In [ ]:
C = rand(basis.l, basis.l);

In [ ]:
my_tu = transform_twobody(my_u, C);

In [ ]:
py_tu = py"transform_two_body_elements"(py_u, C);

In [ ]:
my_tu ≈ py_tu

### Trapz

In [ ]:
f = rand(100);
x = rand(100);

In [ ]:
my_trapz = trapz(f, x);

In [ ]:
py_trapz = py"_trapz"(f, x);

In [ ]:
my_trapz ≈ py_trapz

### Hartree-Fock

In [ ]:
function getP(C, n, l)
    P = zero(C)
    for a in 1:l
        for b in 1:l
            @inbounds P[b, a] = 0
        end
    end
    
    for i in 1:n
        for a in 1:l
            for b in 1:l
                @inbounds P[b, a] += conj(C[a, i]) * C[b, i]
            end
        end
    end
    return P
end

function getF(P, n, l, h, u)
    F = zero(P)
    F .= h
    for c in 1:l
        for d in 1:l
            @inbounds P_dc = P[d, c]
            for a in 1:l
                for b in 1:l
                    @inbounds F[a, b] += P_dc * u[a, c, b, d]
                end
            end
        end
    end
    return F
end

In [ ]:
C = Matrix{Float64}(la.I(basis.l));

### P

In [ ]:
my_P = getP(C, n, basis.l);

In [ ]:
py_P = py"getP"(C, n);

In [ ]:
my_P ≈ py_P

### F

In [ ]:
my_F = getF(my_P, n, basis.l, h, my_u);

In [ ]:
py_F = py"getF"(py_P, h, py_u, n);

In [ ]:
my_F ≈ py_F

### HF-Update

In [ ]:
my_C2 = la.eigvecs(my_F);

In [ ]:
py_C2 = py"np.linalg.eigh"(py_F)[2];

In [ ]:
abs.(my_C2) ≈ abs.(py_C2) # The signs of the eigenvectors might not align

In [ ]:
my_P2 = getP(my_C2, n, basis.l);
my_F2 = getF(my_P2, n, basis.l, h, my_u);
my_C3 = la.eigvecs(my_F2);

In [ ]:
py_P2 = py"getP"(py_C2, n);
py_F2 = py"getF"(py_P2, h, py_u, n);
py_C3 = py"np.linalg.eigh"(py_F2)[2];

In [ ]:
my_P2 ≈ py_P2

In [ ]:
my_F2 ≈ py_F2

In [ ]:
abs.(my_C3) ≈ abs.(py_C3)

### Spfs

In [78]:
py"""
def ho_function(x, n, omega):
    return (
        normalization(n, omega)
        * np.exp(-0.5 * omega * x ** 2)
        * spec.hermite(n)(np.sqrt(omega) * x)
    )

def normalization(n, omega):
    return (
        1.0
        / np.sqrt(2 ** n * spec.factorial(n))
        * (omega / np.pi) ** 0.25
    )
"""

In [84]:
x = Vector(grid);

In [86]:
println(py"ho_function"(x, 2, ω) ≈ spfs[3])

true


### DIIS

In [139]:
n = 6
max_vecs = 10
shape = (n, n, n, n)

(6, 6, 6, 6)

In [140]:
using Random: MersenneTwister

In [141]:
rng = MersenneTwister(0)
vecs = 60
trials = [rand(rng, Float64, shape) for i in 1:vecs]
directions = [rand(rng, Float64, shape) for i in 1:vecs]
errors = [rand(rng, Float64, shape) for i in 1:vecs];

In [142]:
py"""
def mix(max_vecs, trials, directions, errors):
    mixer = DIIS(np, max_vecs)

    result = []
    for i,j,k in zip(trials, directions, errors):
        result.append( mixer.compute_new_vector(i, j, k) )

    return result
"""

In [143]:
function mix_test(max_vecs, trials, directions, errors)
    mixer = DIIS(size(trials[1]), max_vecs);
    
    result = [zero(i) for i in trials]
    for (iter, (i,j,k)) in enumerate( zip(trials, directions, errors) )
        result[iter] .= compute_new_vector(mixer, i, j, k)
    end
    return result
end

mix_test (generic function with 1 method)

In [146]:
py"mix"(max_vecs, trials, directions, errors) ≈ mix_test(max_vecs, trials, directions, errors)

true